# L3: Multi-agent Customer Support Automation
In this lesson, you will learn about the six key elements which help make Agents perform even better:

- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew

import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'